# Proyecto Sprint 11: Aprendizaje Automático en Negocios

## Introducción

Para el presente proyecto, trabajamos en la compañía de extracción de petróleo OilyGiant. El objetivo del mismo es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.


## Inicialización

Iniciamos con la importación de las librerías requeridas para el proyecto.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

pd.options.mode.chained_assignment = None #Esta opcion deshabilita el warning al estandarizar los datos

## Carga de Datos

Continuamos con la lectura de nuestros dataset, revisando que el separador por defecto sea efectivamente el que necesitamos. Para ello, imprimimos una muestra.

In [2]:
geo_0 = pd.read_csv("/datasets/geo_data_0.csv")
geo_1 = pd.read_csv("/datasets/geo_data_1.csv")
geo_2 = pd.read_csv("/datasets/geo_data_2.csv")
geo_0.info()
geo_0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [3]:
geo_1.info()
geo_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [4]:
geo_2.info()
geo_2.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


A nivel general se aprecia que los 3 dataset cargados están bien estructurados. El separador por defecto es la coma y sus columnas están nombradas con minúsculas. Tampoco se aprecian datos faltantes en ninguna de las mismas.

## Preparación de los datos

### Tratado de valores duplicados

Revisaremos si en nuestros dataset existen valores que estén repetidos (a nivel de filas).

In [5]:
geo_0.duplicated().sum()

0

In [6]:
geo_1.duplicated().sum()

0

In [7]:
geo_2.duplicated().sum()

0

Ya que no existen valores ausentes ni duplicados continuaremos con la segmentación del dataset.

### Segmentación del Dataset

Comenzamos definiendo los features y target de cada dataset.

In [8]:
features0 = geo_0.drop("product", axis = 1)
target0 = geo_0["product"]

features1 = geo_1.drop("product", axis = 1)
target1 = geo_1["product"]

features2 = geo_2.drop("product", axis = 1)
target2 = geo_2["product"]

Para el dataset de entrenamiento dejaremos 75% del dataframe total, para el de validación un 25% de los datos. Esto, para nuestro primer dataframe geo_0.

In [9]:
features0_train, features0_valid, target0_train, target0_valid = train_test_split(
    features0, target0, test_size=0.25, random_state=1702)

### Estandarización de datos

In [10]:
numeric = ["f0", "f1", "f2"]
scaler = StandardScaler()
scaler.fit(feateures0_train[numeric])
features0_train[numeric] = scaler.transform(features0_train[numeric])
features0_valid[numeric] = scaler.transform(features0_valid[numeric])

## Entrenamiento del Modelo

### Modelo para el Dataset geo_0

In [11]:
model = LinearRegression()
model.fit(features0_train[numeric], target0_train) # entrena el modelo en el conjunto de entrenamiento
predictions0_valid = model.predict(features0_valid[numeric])

result = mean_squared_error(target0_valid,predictions0_valid)**0.5

print("Volumen medio predicho", predictions0_valid.mean().round(2))
print("RMSE del modelo es:", result.round(2))
print("Volumen medio real",target0.mean().round(2))

Volumen medio predicho 92.47
RMSE del modelo es: 37.88
Volumen medio real 92.5


Resultado no satisfactorio acorde a los rangos de producto en nuestro dataset. El RMSE es demasiado alto, por ende el error no nos permite asegurar que sea un buen modelo. Hay que comparar las otras regiones. Sus volumenes son muy similares, por lo que solo el RMSE señala que el modelo tiene alta tasa de errores.

### Función para dataset geo_1 y geo_2

In [12]:
current_predictions = []
def dataprep(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                                  test_size = 0.25, 
                                                                                  random_state = 1702)
    scaler = StandardScaler()
    features_train.drop("id", axis = 1, inplace = True) #para evitar errores al modelar dejamos solo col numericas
    features_valid.drop("id", axis = 1, inplace = True) #para evitar errores al modelar
    scaler.fit(features_train)
    features_train = scaler.transform(features_train)
    features_valid = scaler.transform(features_valid)
    model = LinearRegression()
    model.fit(features_train, target_train)
    global predictions_valid #la hacemos global para utilizarla en la funcion de ganancias
    predictions_valid = model.predict(features_valid)
    current_predictions = pd.Series(predictions_valid)
    result = mean_squared_error(target_valid, predictions_valid)**0.5
    
    print("Volumen medio predicho", predictions_valid.mean().round(2))
    print("RMSE del modelo es:", result.round(2))

### Modelo para el Dataset geo_1

In [13]:
dataprep(features1, target1)
print("Volumen medio real",target1.mean().round(2))

Volumen medio predicho 68.44
RMSE del modelo es: 0.89
Volumen medio real 68.83


Los volumenes medios predicho y real son similares y el RMSE es bajo. Esta es la mejor región acorde a los datos que tenemos hasta ahora. El modelo funciona acorde a lo que uno esperaría.

### Modelo para el Dataset geo_2

In [14]:
dataprep(features2, target2)
print("Volumen medio real",target2.mean().round(2))

Volumen medio predicho 95.15
RMSE del modelo es: 39.98
Volumen medio real 95.0


Mismo caso que dataset geo_0. Sus volumenes predichos y reales en la media son iguales. Sin embargo su RMSE es demasiado alto para considerarlo fiable.

### Cálculo de Ganancias

Nos señalan que dada la inversión de 100 millones por 200 pozos petrolíferos, de media un pozo petrolífero debe producir al menos un valor de 500,000 dólares en unidades para evitar pérdidas (esto es equivalente a 111.1 unidades). 

Por ello, lo que debemos efectuar es comparar esta cantidad con la cantidad media de reservas en cada región.
Para referirnos a las regiones las clasificaremos acorde a su valor de dataset. Es decir, región 0, 1 y 2 respectivamente.


In [15]:
print("Media region 0:", geo_0["product"].mean().round(2))
print("Media region 1:", geo_1["product"].mean().round(2))
print("Media region 2:", geo_2["product"].mean().round(2))


Media region 0: 92.5
Media region 1: 68.83
Media region 2: 95.0


Claramente ninguna región cumple el requisito de forma inicial, ya que como vimos previamente, tenemos valores donde el mínimo es 0, por ende aquello baja el promedio de la región en su totalidad. Lo que nos señala el enunciado que para evitar pérdidas deberían estar alrededor de 111.1 unidades al menos.

Ahora, nos piden que para 200 pozos petrolíferos produzcan lo señalado para, al menos, evitar pérdidas, por ende lo que se debe efectuar es seleccionar los 200 mejores pozos por región y evaluar lo respectivo con aquellos nuevos datos.

#### Ganancias por región

Definimos la función para calcular las ganancias respectivas.

In [16]:
def ganancias(features, target):
    dataprep(features, target)
    global predictions #para utilizarla en bootstrapping
    predictions = pd.Series(predictions_valid).sort_values(ascending = False).head(200)
    ganancias = predictions.sum()*4500
    print("ganancias predichas de la region (millones de dolares) =", (ganancias/1000000).round(2))



#### Ganancia región 0

Considerando sus mejores 200 pozos en la región.

In [17]:
ganancias(features0, target0)


Volumen medio predicho 92.47
RMSE del modelo es: 37.88
ganancias predichas de la region (millones de dolares) = 138.58


La ganancia de la región 0 es bastante alta (la mayor de las 3 regiones). Sin embargo, algo que no podemos pasar por alto es su valor elevado de RMSE.

#### Ganancia región 1

Considerando sus mejores 200 pozos en la región.

In [18]:
ganancias(features1, target1)

Volumen medio predicho 68.44
RMSE del modelo es: 0.89
ganancias predichas de la region (millones de dolares) = 124.91


Para maximizar la ganancia, el RMSE del modelo debe ser bajo, por ende nos conviene tomar los pozos de la región 1. Esta es la que en volumen medio tenia la menor cantidad, sí, y también es la que posee menor ganancia, pero acorde a lo planteado en las condiciones, cumple con tener el valor arriba de lo requerido. Por ende sus ganancias proyectadas estarían más cerca de lo real.

#### Ganancia región 2

Considerando sus mejores 200 pozos en la región.

In [19]:
ganancias(features2, target2)

Volumen medio predicho 95.15
RMSE del modelo es: 39.98
ganancias predichas de la region (millones de dolares) = 134.26


Mismo caso que la region 0. Mejor ganancia que la región 1, pero el RMSE es demasiado elevado.

### Ganancias y Riesgos por Región

Utilizando las predicciones que se almacenaron en el paso anterior, se emplea la técnica del bootstrapping con 1000 muestras para hallar la distribución de los beneficios.

#### Región 0

In [20]:
ganancias(features0, target0) #para obtener el valor de predictions correspondiente a esta región

Volumen medio predicho 92.47
RMSE del modelo es: 37.88
ganancias predichas de la region (millones de dolares) = 138.58


In [21]:
state = np.random.RandomState(1702) #para obtener diferentes submuestras

values = []
def conf_int(prediction):
    values = []
    for i in range(1000):
        subsample = prediction.sample(frac = 1, replace = True,random_state = state) #subsample con reemplazo 
        values.append(subsample.quantile(1))

    values= pd.Series(values)
    print('Media de las submuestras:', values.mean().round(2))
    print("Ganancia Promedio Individual submuestras (miles de dolares):", (values.mean()*4.5).round(2))
    
    confidence_interval = st.t.interval(0.95, len(values)-1, loc=values.mean(), scale=values.sem())

    print('Intervalo de confianza del 95 % submuestras:', confidence_interval)
    print('Ganancia potencial (millones de dolares):', ((values.mean()*4500)*200/1000000).round(2)  )
    
conf_int(predictions)

Media de las submuestras: 181.78
Ganancia Promedio Individual submuestras (miles de dolares): 818.03
Intervalo de confianza del 95 % submuestras: (181.4954882714561, 182.07240229604986)
Ganancia potencial (millones de dolares): 163.61


#### Región 1


In [22]:
ganancias(features1, target1)
print()
conf_int(predictions)

Volumen medio predicho 68.44
RMSE del modelo es: 0.89
ganancias predichas de la region (millones de dolares) = 124.91

Media de las submuestras: 139.86
Ganancia Promedio Individual submuestras (miles de dolares): 629.37
Intervalo de confianza del 95 % submuestras: (139.85000119474157, 139.8695212095075)
Ganancia potencial (millones de dolares): 125.87


#### Región 2


In [23]:
ganancias(features2, target2)
print()
conf_int(predictions)

Volumen medio predicho 95.15
RMSE del modelo es: 39.98
ganancias predichas de la region (millones de dolares) = 134.26

Media de las submuestras: 172.42
Ganancia Promedio Individual submuestras (miles de dolares): 775.87
Intervalo de confianza del 95 % submuestras: (172.22802759270837, 172.6033695504491)
Ganancia potencial (millones de dolares): 155.17


#### Conclusiones

Acorde a lo que hemos visto, claramente las regiones 0 y 2 son superiores a niveles de ganancia potencial y en nuestros resultados de un inicio presentan mejores valores promedio para los pozos. 

Sin embargo, lo que sigue causando ruido (a mi parecer) es su valor de RMSE elevado al evaluar los modelos en ambas regiones. 

Considerando lo anterior, mi región para la inversión sería la 1, ya que el modelo se comporta mejor desde un inicio y con los valores evaluados mantenemos ganancias positivas que nos llevan a que al elegir sus mejores 200 pozos sea rentable.